# Cut different FOVs from existing simulated data

In [ ]:
import json
from itertools import product

import numpy as np
from sim_util import save_as_sequence, load_tiff_sequence

def sim_from_definition_existing_images(def_path, save_fsting_new=None):
    
    # load settings
    with open(def_path, 'r') as fd:
        params = json.load(fd)
        
    save_fstring = params['save_fstring']
    seed = params['seed']
    raw_data_path = params['raw_data_path']
    raw_data_dims = params['raw_data_dims']
    downsampling = params['downsampling']
    phys_dims = params['phys_dims']
    na_illum = params['na_illum']
    na_detect = params['na_detect']
    ri_medium =  params['ri_medium']
    ri_delta_range = params['ri_delta_range']
    two_sided_illum = params['two_sided_illum']
    lambdas = params['lambdas']
    fov_size = params['fov_size']
    point_fov_size = params['point_fov_size']
    n_points_per_fov = params['n_points_per_fov']
    points_min_distance = params['points_min_distance']
    min_off = params['min_off']
    max_off = params['max_off']
    x_locs = params['x_locs']
    y_locs = params['y_locs']
    z_locs = params['z_locs']
    fields = params['fields']
    padding = params['padding']
    conv_subblocks = params['conv_subblocks']
    bprop_nvolumes = params['bprop_nvolumes']

    for lam, right_illum in product(lambdas, ([False] if not two_sided_illum else [True, False])):
        for xi in range(len(x_locs)):

            # load the previously generated "all" images
            out_dir_all = save_fstring.format(x=xi, y='all', z='all', lam=lam, illum=(1 if not right_illum else 2))
            out_signal = load_tiff_sequence(out_dir_all, 'bbeam_sim_c1.*?')
            out_desc = load_tiff_sequence(out_dir_all, 'bbeam_sim_c2.*?')

            yidx, zidx = np.meshgrid(range(len(y_locs)), range(len(z_locs)))    
            for yi, zi in zip(yidx.flat, zidx.flat):

                field_info = fields[','.join(map(str, (xi, yi, zi)))]
                off = field_info['off']
                loc = [z_locs[zi], y_locs[yi], x_locs[xi]]
                min_ = np.array(loc) + np.array(off) - np.ceil(np.array(fov_size) / 2)
                max_ = np.array(loc) + np.array(off) + np.floor(np.array(fov_size) / 2)

                # out dir
                out_dir = (save_fstring if save_fsting_new is None else save_fsting_new).format(x=xi, y=yi, z=zi, lam=lam, illum=(1 if not right_illum else 2))

                # cut signal and descriptors
                # NB: downsampling
                min_ = min_//downsampling
                max_ = max_//downsampling

                idxs = tuple([slice(int(mi), int(ma)) for mi, ma in zip(min_, max_)])

                # save
                save_as_sequence(out_signal[idxs], out_dir, file_pattern='bbeam_sim_c1_z{plane}.tif')
                save_as_sequence(out_desc[idxs], out_dir, file_pattern='bbeam_sim_c2_z{plane}.tif')
                
                
sim_from_definition_existing_images('/Volumes/davidh-ssd/mv-sim/sim6/sim6_bigFOV.json',
                                    "/Volumes/davidh-ssd/mv-sim/sim62/sim6-bbeam-x{x}-y{y}-z{z}-i{illum}-l{lam}/")